In [ ]:
import numpy as np
import pandas as pd
import os
from posixpath import join
import yaml



In [ ]:
def ecvl_yaml(name, description, filenames, labels, train_ids, valid_ids, test_ids):
    d = {
        "name"        : name,
        "description" : description,
        "classes"     : [], 
        "images"      : [],
        "split"       : dict(training = train_ids, 
                            validation = valid_ids, 
                            test=test_ids)
    }
    imgs = []
    for fn, l in zip(filenames, labels):
        imgs.append({
            "location": fn,
            "label": l
        })
    d["images"] = imgs
    d["classes"] = sorted(list(set(labels)))
    return d

In [ ]:
# mimic normal vs rest, unbalanced
# 3 models, dataset stays the same, random seed changes

in_file = join("/mnt/datasets/mimic-cxr/training_data/mimic", "normal_bin_unbal.tsv")  # input dataset
exp_fld = "/mnt/datasets/uc5/UC5_pipeline_forked/experiments_eddl/mimic/normal_unbal"  # output fld

img_fld = "/mnt/datasets/mimic-cxr/mimic-cxr-jpg/physionet.org/files/mimic-cxr-jpg/2.0.0"

dataset = pd.read_csv(in_file, sep="\t")
def adjust_path(path):
    path = path[:-len(".dcm")]
    path = join(img_fld, path + ".jpg")
    return path
display(dataset.T)

dataset["path"] = dataset["path"].apply(lambda path: adjust_path(path))
dataset.set_index(pd.RangeIndex(len(dataset)), inplace=True)
splits = ["train", "validate", "test"]
filenames = []
labels = []
ids = {}
for s in splits:
    split = dataset.loc[dataset.split == s]
    print(f"{s}: {split.shape}")
    filenames += split.path.tolist()
    labels += split.target.tolist()
    ids[s] = split.index.values.tolist()
print(f"filenames, {len(filenames)}")
print(f"labels, {len(labels)}")

yml_ds = ecvl_yaml(filenames, labels, ids["train"], ids["validate"], ids["test"])
for k, v in yml_ds.items():
    print(f"{k}: {len(v)}")

n_folds = 3
for i in range(n_folds):
    out_fld = join(  exp_fld, f"fold_{i}" )
    os.makedirs(out_fld, exist_ok=True)
    fn = join(out_fld, "dataset.yml")
    
    with open(fn, "w") as fout:
        yaml.safe_dump(yml_ds, fout, default_flow_style=None)
    print(f"saved {fn}")
#     # takes up to 

In [ ]:
a = "ciao"
a[:-(len("ao"))]